In [1]:
import torch
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
# XOR 문제를 풀기 위한 입력과 출력을 정의해줍니다.

X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [6]:
# 이제 다층 퍼셉트론을 설계합니다. 아래는 입력층, 은닉층1, 은닉층2, 은닉층3, 출력층을 가지는 은닉층이 3개인 인공 신경망입니다.

model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.Sigmoid()
          ).to(device)

# 그림으로 표현하면 2개 원 -> 10개원 -> 10개원 -> 10개원 -> 1개원 각각 레벨마다 전부 연결

In [7]:
# 이제 비용 함수와 옵타마이저를 선언합니다. nn.BCELoss()는 이진 분류에서 사용하는 크로스엔트로피 함수입니다.

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

In [8]:
# 총 10,001번의 에포크를 수행합니다. 각 에포크마다 역전파가 수행된다고 보면 되겠습니다.

for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

0 0.7408947944641113
100 0.6931389570236206
200 0.6931374669075012
300 0.6931357383728027
400 0.6931341886520386
500 0.6931324601173401
600 0.693130612373352
700 0.6931287050247192
800 0.6931267976760864
900 0.6931246519088745
1000 0.6931225061416626
1100 0.693120002746582
1200 0.6931174993515015
1300 0.6931147575378418
1400 0.693111777305603
1500 0.6931085586547852
1600 0.6931051015853882
1700 0.6931012868881226
1800 0.6930969953536987
1900 0.6930922269821167
2000 0.693087100982666
2100 0.693081259727478
2200 0.693074643611908
2300 0.6930671334266663
2400 0.6930586099624634
2500 0.6930486559867859
2600 0.693037211894989
2700 0.6930238008499146
2800 0.6930078268051147
2900 0.6929886937141418
3000 0.6929655075073242
3100 0.6929367780685425
3200 0.6929006576538086
3300 0.6928542852401733
3400 0.6927931904792786
3500 0.692710280418396
3600 0.6925929188728333
3700 0.6924186944961548
3800 0.6921423673629761
3900 0.6916618347167969
4000 0.6907041072845459
4100 0.6883158683776855
4200 0.67906

In [9]:
# 이제 모델이 XOR 문제를 풀 수 있는지 테스트 해봅시다.

with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[6.8926340e-05]
 [9.9988186e-01]
 [9.9989271e-01]
 [1.5691276e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0


실제값은 0, 1, 1, 0이며 예측값은 0, 1, 1, 0으로 문제를 해결하는 모습을 보여줍니다.

단층으로는 불가능한것을 다층을 이용하여 해결하였습니다.